In [1]:
import math
from sympy.solvers import solve
from sympy import Symbol
import ipywidgets as widgets
import ipysheet


In [5]:
sheet = ipysheet.sheet(rows=10, columns=2, column_headers=False, row_headers=False)
cell_odd_io = ipysheet.cell(0, 1,  label_left='ODD IO', read_only=True)
cell_odd_entrada_365 = ipysheet.cell(1, 1, label_left='odd entrada 365', read_only=True)
cell_val_entrada_io = ipysheet.cell(2, 1,  label_left='valor entrada IO')
cell_val_entrada_365 = ipysheet.cell(3, 1, label_left='valor entrada 365', color='white', background_color='gray' ,read_only=True)
cell_odd_saida_sucesso_365 = ipysheet.cell(4, 1,  label_left='odd saida sucesso 365', color='white', background_color='green', read_only=True)
cell_odd_saida_falha_365 = ipysheet.cell(5, 1,  label_left='odd saida falha 365', color='white', background_color='red', read_only=True)
cell_stake = ipysheet.cell(6, 1, 200, label_left='stake')
cell_goal = ipysheet.cell(7, 1, 0.1, label_left='lucro(%)', read_only=True)
cell_fail = ipysheet.cell(8, 1, 0.25, label_left='prejuizo(%)', read_only=True)
cell_confianca = ipysheet.cell(9, 1, 2, label_left='indice confianca', read_only=True)

# create a slider linked to cell a
slider = widgets.FloatSlider(min=1.4, max=1.7, step=0.01, description='odd IO')
stake_atual = widgets.IntSlider(min=200, max=1000, step=10, description='stake')
goal = widgets.FloatSlider(min=0, max=0.5, step=0.05, description='lucro(%)')
fail = widgets.FloatSlider(min=0, max=0.5, step=0.05, description='preju(%)')
confianca = widgets.FloatSlider(min=1.95, max=2.1, step=0.01, description='indice')
widgets.jslink((cell_odd_io, 'value'), (slider, 'value'))
widgets.jslink((cell_stake, 'value'), (stake_atual, 'value'))
widgets.jslink((cell_goal, 'value'), (goal, 'value'))
widgets.jslink((cell_fail, 'value'), (fail, 'value'))
widgets.jslink((cell_confianca, 'value'), (confianca, 'value'))

# changes in a or b should trigger this function
def entrada_365(change):
    cell_val_entrada_365.value = cell_stake.value - cell_val_entrada_io.value
def valor_entrada_io(change):
    cell_val_entrada_io.value = math.ceil(((1+cell_goal.value)*cell_stake.value)/(cell_odd_io.value))

def odd_saida_365(change):
    cell_odd_saida_sucesso_365.value = ((1+cell_goal.value)*cell_stake.value)/cell_val_entrada_365.value
    cell_odd_saida_falha_365.value = ((1-cell_fail.value)*cell_stake.value)/cell_val_entrada_365.value

def get_odd_365_confianca_2(odd_io):
    x = Symbol('x')
    alpha = ((x-odd_io)/(x+odd_io))
    result = solve(x*(1-alpha) - cell_confianca.value, x)
    return(max(result))

    
def odd_entrada_365(change):
    cell_odd_entrada_365.value = get_odd_365_confianca_2(cell_odd_io.value)

cell_val_entrada_io.observe(entrada_365, 'value')


cell_odd_io.observe(valor_entrada_io, 'value')
cell_odd_io.observe(odd_entrada_365, 'value')

cell_val_entrada_365.observe(odd_saida_365, 'value')

widgets.VBox([sheet, slider, stake_atual,goal, fail, confianca])